In [ ]:
%%capture
%pip install fast-explain
%pip install lightgbm

In [ ]:
from FastExplain import *
df = load_titanic_data()

In [ ]:
df.head()

# Fit Data

In [ ]:
help(model_data)

In [ ]:
model = model_data(df, dep_var="Survived", model="ebm")

# View Data

View how training data is split and how variables are transformed

In [ ]:
model.data.train_xs.head()

Ordinal encoding is applied to categorical columns by default. View their mapping

In [ ]:
model.data.cat_mapping

Plot correlation between variables

In [ ]:
model.feature_correlation(plotsize = (1000,500))

# Model Info

Get model object

In [ ]:
model.m

Get model parameters

In [ ]:
model.params

Get fit error. Benchmark is defined as the average of training

In [ ]:
model.error

Plot feature importance and highlight certain features

In [ ]:
model.plot_feature_importance(feature_highlights=["Age"])

# Explore Fit

Simple one way analysis

In [ ]:
help(model.plot_one_way_analysis)

In [ ]:
model.plot_one_way_analysis("Age")

In [ ]:
model.plot_one_way_analysis("Sex")

ALE Plots

In [ ]:
help(model.plot_ale)

In [ ]:
model.plot_ale("Age")

In [ ]:
model.plot_ale("Sex")

In [ ]:
help(model.plot_ebm_explain)

In [ ]:
model.plot_ebm_explain("Age")

In [ ]:
model.plot_ebm_explain("Sex")

SHAP Values

In [ ]:
model = model_data(df, dep_var="Survived", model="rf")

In [ ]:
model.get_shap_values()

In [ ]:
model.shap_dependence_plot("Age")

In [ ]:
model.shap_dependence_plot("Sex")

# Support Custom Models

In [ ]:
from lightgbm import LGBMClassifier
model = model_data(df, 'Survived', model=LGBMClassifier)

In [ ]:
model.plot_ale("Age")

In [ ]:
model.shap_dependence_plot("Age")